In [1]:
!pip install MEGABYTE-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.2/720.2 kB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 6.3 MB/s eta 0:00:00


In [ ]:
import torch
from MEGABYTE_pytorch import MEGABYTE


model = MEGABYTE(
    num_tokens = 8000,             # number of tokens
    dim = (256, 256),               # transformer model dimension (512 for coarsest, 256 for fine in this example)
    max_seq_len = (512, 4),        # sequence length for global and then local. this can be more than 2
    depth = (6, 4),                 # number of layers for global and then local. this can be more than 2, but length must match the max_seq_len's
    dim_head = 64,                  # dimension per head
    heads = 8,                      # number of attention heads
    flash_attn = True               # use flash attention
)

x = torch.randint(0, 8000, (1, 512, 4))

loss = model(x, return_loss = True)
loss.backward()

# then after much training

logits = model(x)

# and sample from the logits accordingly
# or you can use the generate function

sampled = model.generate(temperature = 0.9, filter_thres = 0.9) # (1, 512, 4)

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda


100%|██████████| 2048/2048 [11:43<00:00,  2.91it/s]


In [2]:
%cd /content/drive/MyDrive/Colab_Notebooks

/content/drive/MyDrive/Colab_Notebooks


In [ ]:
!git clone https://github.com/lucidrains/MEGABYTE-pytorch.git

Cloning into 'MEGABYTE-pytorch'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 127 (delta 46), reused 41 (delta 34), pack-reused 67
Receiving objects: 100% (127/127), 35.31 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (79/79), done.


In [3]:
%cd MEGABYTE-pytorch 

/content/drive/MyDrive/Colab_Notebooks/MEGABYTE-pytorch


In [4]:
!python train.py

Non-A100 GPU detected, using math or mem efficient attention if input tensor is on cuda
training:   0% 0/100 [00:00<?, ?it/s]training loss: 5.713071823120117
validation loss: 5.336212158203125
training loss: 5.33056640625
training loss: 5.09693717956543
training:   3% 3/100 [00:10<05:25,  3.35s/it]training loss: 4.7716264724731445
training loss: 4.6202287673950195
training loss: 4.4048309326171875
training loss: 4.325751781463623
training loss: 4.313610076904297
training:   8% 8/100 [00:22<04:09,  2.71s/it]training loss: 4.081308364868164
training loss: 3.9244985580444336
training loss: 4.142832279205322
training loss: 3.8375444412231445
training loss: 3.88970947265625
training:  13% 13/100 [00:34<03:46,  2.60s/it]training loss: 3.8636796474456787
training loss: 3.9575092792510986
training loss: 3.8612284660339355
training loss: 3.812952995300293
training loss: 3.762855291366577
training:  18% 18/100 [00:47<03:31,  2.58s/it]training loss: 3.751217842102051
training loss: 3.587889909744

In [9]:
from MEGABYTE_pytorch import MEGABYTE
import random
import gzip
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset


def decode_token(token):
    return str(chr(max(32, token)))


def decode_tokens(tokens):
    return ''.join(list(map(decode_token, tokens)))


model = MEGABYTE(
    num_tokens = 256,
    dim = (768, 512, 256),
    depth = (6, 4, 2),
    max_seq_len = (512, 4, 4),
    flash_attn = True
).cuda()


with gzip.open('./data/enwik8.gz') as file:
    x = np.frombuffer(file.read(int(95e6)), dtype=np.uint8).copy()
    train_x, valid_x = np.split(x, [int(90e6)])
    data_train, data_val = map(torch.from_numpy, (train_x, valid_x))


class TextSamplerDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):
        rand_start = torch.randint(0, self.data.size(0) - self.seq_len, (1,))
        full_seq = self.data[rand_start: rand_start + self.seq_len].long()
        return full_seq.cuda()

    def __len__(self):
        return self.data.size(0) // self.seq_len


SEQ_LEN = 8192
PRIME_LEN = 100
val_dataset   = TextSamplerDataset(data_val, SEQ_LEN)
inp = random.choice(val_dataset)[:-1]
prime_inp = inp[:PRIME_LEN]
prime = decode_tokens(prime_inp)
print(f'%s \n\n %s', (prime, '*' * 100))
sample = model.generate(prime_inp[None, :])
sample = sample.flatten(1)
output_str = decode_tokens(sample[0][PRIME_LEN:])
print(output_str)

%s 

 %s ("l have a male heir despite his wife's previous failed pregnancies (one stillbirth, one miscarriage a", '****************************************************************************************************')


100%|██████████| 8092/8092 [04:20<00:00, 31.10it/s]


G! tã2î  Tç !°Ýíì & ^ÄQ Ó-} ÿoÅÂý J o ·h ª W© 2r}\I=r· E1  ¦Ln»}0= aV.Ä}yHÉ;/ë}¼HD dWÄÎKµëÿeÃ ´ ßS¡  \µþAEô £Ó òëòÈ3 GCË) ¯Ó   µeDÂ~ûø«K ç Glâ¿+àNÂ  ¦T&µ0ä!¦ µlÿ ºµ/rGþ} [Ý V[IÉ¦Hh©ú6õUVHÎ¦(Ó} >Sõãó|6¯2¢6¿2¤¦òÞ 2ëQròl¦¿5Ý"á:I=!8À´  CöI[b¥Ùf/ròílÖ²ßÞÉÝ}æ ;/;h²  Ä/¦²ýÿrG'Àß ì«!Î}ª8¤9ë  Æ >ªrõe¾µ» Aë'~Ó ´Z ¦½b/Â ß= 'eÏ Éü!=É  EK~t¥.C  Þßm ó! ´ÀU·¤ Ýr)\ëåoVMµï~XßRßå¥M[eª;f¢R YË}òh£I= µß|¥r²DQl" fWÉcå½âryî¦ßXîÿÀß ÉÀû´Àîí¹Î· Ü  Þ'§  ·LRÄK^)ôóO®È2g ¾ßwDr²î/;l û ¸Ëe¿W£r¿ÃÆ5ÓXV·l}) Ó |6½}ôßü6¯/\5ê Qçü}ßwYâæXz© eªn OU,Tô µe£ 8Sj5Kµ=  í}¿ µ;cålPÿHûl  R^HÑ©  À}úò ÿr ô¤ ~å á ÓâK ) ¹Ä? úÇGmí 5[`) á  æ-r/ô  Ä°ÆfµVô°=À[OÉ ²û¡º Ó=ª8+îTcNôÃSÞ¦n/Þ¿ Ó´¯G ràýoÀáë' Gà} ÀU:2ÀÖc°ÑÕ.¨Ñ åol  $eò  ^©re!/cIÉ .ût .¨=Ç5âÉcb)¥Ñ}r]¤©ÇeÕ Ó ÷UÀ·YºKÝ6ô}Aÿ hi xÎ^ýÞ¯þAç¯¤ ¯·¤ 8WçYòlÉ 5 É }µG!Ä  ÜYç þN¯þµÄÖp«ÉÀ&Ó Ú· Óc8=å2£  ôk¿5 âúK¥ ^])Â¹´¼Nr³ $D"3ÓÉXüS rG=ß5Óâ`mÞr¼õ/ ãüÄL  í­nù.Ô ¢ ÇÐVÂ >ªÀÖµ» 6e!¿ ç  $Æ8ýÿr¼O .  áÝWDÆÄÂ .Ð¦û

In [25]:
def encode(list_of_strings, pad_token_id=0):
    max_length = max([len(string) for string in list_of_strings])

    # create emtpy tensors
    attention_masks = torch.zeros((len(list_of_strings), max_length), dtype=torch.long)
    input_ids = torch.full((len(list_of_strings), max_length), pad_token_id, dtype=torch.long)

    for idx, string in enumerate(list_of_strings):
        # make sure string is in byte format
        if not isinstance(string, bytes):
            string = str.encode(string)

        input_ids[idx, :len(string)] = torch.tensor([x for x in string])
        attention_masks[idx, :len(string)] = 1

    return input_ids.cuda(), attention_masks


encoded, attention_masks = encode(["In 1965, Brooks left IBM to found the Department of"])
prime_encoded = encoded[0]
prime = decode_tokens(prime_encoded)
print(f'%s \n\n %s', (prime, '*' * 100))
sample_input = model.generate(prime_encoded[None, :])
sample_input = sample_input.flatten(1)
output = decode_tokens(sample_input[0][PRIME_LEN:])
print(output)

%s 

 %s ('In 1965, Brooks left IBM to found the Department of', '****************************************************************************************************')


100%|██████████| 8141/8141 [04:22<00:00, 31.06it/s]


 ô ==ª2 Ç¯À°X Eú   ¥\µë/ &²á b-rnoVògÄ  ºU"G[ ø\8ûÄ KÜ=?çýx  áÿ "´ KÌ/ÀhÞ»NlÄrÓ£ úCÞ^cwÎ"g  ÞÀPJW}ãþ"ÆÒç  ßvKü6Ä²h=r ª©.Cì¼¿ÇòhK©/Y}h \wª DZ} pSÿ@ãíçrn°9ÖTå¥`2úÿ ßmÿÜÝÉ MNa^²Ó-À&ÑÃN çÂºx $ ç.r}ò bl!S}µþl +LxÿHí }û;µÓÿ ·l}ÿß ÿ  |ñ«åür& }É´Ð À.Íp hÀ >ß  Ü }ypÐ ¿  Ã [î æû  x)r&Àí»!xÀ[bØP§´  |l§KÌÉ >ÞÂ~\¿òWR 6Dñ$Ù) ·Ï}). 6¦HûUÈè  ·/¿ r rÚ¼´ 2h×öMÓ¡ ÉÀt ËWü]Z©ÈÐXxßò À[ITW)\Gû ºÉ CÞ,i}rà(¿°}µ»b£À·ô ¸=¦3² ! b ÂcÑëVHù þÄt ·Ð Éµ¤¢ÿHÎ cxÇ Gb¥ [ë/ÞFÄ YÛK\¨\KÌ8|ëI:ílºKI}y²û [|ã:e¦ q·L-[ßërÈå !~}ÿ©Óò ®ßÿ GëUI ê}¿  9\Ý}D µrX ¼/òÞ"2ÓoßKÜ=!Xå¥j äÿDS6ßXå°¿KµÉrp°6  l¥ ÝJ«$ÿ@&²á¿ãbÃ cÿµÎ Â·¤ÄÀ3 /¯G|= Kî ZëYßhÉ¦·Â >ì¦I N-r Yÿ þ}© UxÇüÃ Vx È ´úÀ/¨£ EË\c¤U) ÓâI(²â Kß]EøòGë  åÓ£ÂK oV ûÞ  åsZXHUr²l´nlâ  }µ2Ó/ Mµ«zý;ßK²¿8   p²/ôò ëIîÄr.h¡`ãx´áo  ÿµ·ú¦r W)®.Íâ C-)ùøÞ6DÂ ¤)gKìr·¨ æû E £ß ¢ ! ex ålÿDÖÓò\KÜ¤!.}ÃÆ©Cç¼bE 5úÄ ÆbÓyGÀ¸8àte ã·eÂ lö¬YU' 5KÂ·¤ñ© úøòG  ¯2©Ó4 ²QNôV~[¼ÞÉ´eî nëÿ GWøndÐÿyx)¥ò ;ªÞÖlo5ý¦ÈÖÓÿî )ª þ